# Advent of Code 2021

Registered 2021 with GitHub (jhchou)

* [Advent of Code](https://adventofcode.com/)
* [Julialang private leaderboard](https://adventofcode.com/2021/leaderboard/private/view/213962)

## Setup

* created new repository on GitHub (public)
* created new local folder

```
    echo "# 2021_adventofcode" >> README.md
    git init
    git add README.md
    git commit -m "first commit"
    git branch -M main
    git remote add origin https://github.com/jhchou/2021_adventofcode.git
    git push -u origin main
```

* `Jupyter: Create New Jupyter Notebook`
    * selected Julia 1.6.3 for kernel
* `Julia: Start REPL`
    * start with empty environment
    * `] activate .`

## Day 1

To do this, count the number of times a depth measurement increases from the previous measurement. (There is no measurement before the first measurement.)

In [1]:
# Clunky solution, reading file line by line

lastnum = 99999999
count = 0
f = open("day1.txt", "r")
while !eof(f)
    line = readline(f)
    num = parse(Int, line)
    if num > lastnum
        count += 1
    end
    lastnum = num
end
close(f)
count # 1374

1374

### Part 2

Comparing sums over a sliding window

In [2]:
# Reading all the lines at once into a vector of Int, then looping

f = open("day1.txt", "r")
lines = parse.(Int, readlines(f)) # read entire file as vector of Int
close(f)

len = length(lines)
count = 0
windowsize = 3
for i in 1 : (len - windowsize)
    if sum(lines[i+1 : i+windowsize]) > sum(lines[i : i+windowsize-1])
        count += 1
    end
end
count # 1418

1418

In [3]:
# Simplify reading file into vector of Int, then use sum of booleans on list comprehension

lines = parse.(Int, eachline("day1.txt"))
windowsize = 3 # 1 for part 1; 3 for part 2
sum([sum(lines[i+1 : i+windowsize]) > sum(lines[i : i+windowsize-1])] for i in 1 : (length(lines) - windowsize))

1-element Vector{Int64}:
 1418

In [4]:
# With chaining, diff, and >(0)

lines = parse.(Int, eachline("day1.txt"))
windowsize = 3 # 1 for part 1; 3 for part 2

ΣΔ(i) = i |> diff .|> >(0) |> sum |> println

ΣΔ(lines) # Part 1

blocks = [ sum( lines[i : (i+windowsize-1)] ) for i in 1 : (length(lines) - windowsize + 1) ]
ΣΔ(blocks) # Part 2

1374
1418


## Day 2

### Part 1

It seems like the submarine can take a series of commands like forward 1, down 2, or up 3:

* forward X increases the horizontal position by X units.
* down X increases the depth by X units.
* up X decreases the depth by X units.

Your horizontal position and depth both start at 0.

What do you get if you multiply your final horizontal position by your final depth?

In [19]:
depth = horizontal = 0
lines = eachline("day2.txt")
for line in lines
    m = match(r"(.+) (\d+)", line)
    m === nothing && continue
    (command, numstring) = m.captures
    num = parse(Int, numstring)
    if command == "forward"
        horizontal += num
    elseif command == "down"
        depth += num
    elseif command == "up"
        depth -= num
    else
        println("*** ERROR ***")
    end
end
println(horizontal * depth) # 2102357

2102357


### Part 2

In addition to horizontal position and depth, you'll also need to track a third value, aim, which also starts at 0. The commands also mean something entirely different than you first thought:

* down X increases your aim by X units.
* up X decreases your aim by X units.
* forward X does two things:
    * It increases your horizontal position by X units.
    * It increases your depth by your aim multiplied by X.


In [9]:
# Use pipe to avoid intermediary variable

depth = aim = horizontal = 0
lines = eachline("day2.txt")
for line in lines
    m = match(r"(.+) (\d+)", line)
    m === nothing && continue
    (command, num) = m.captures |> captures -> (captures[1], parse(Int, captures[2]))
    if command == "forward"
        horizontal += num
        depth += aim * num # added for part 2
    elseif command == "down"
        aim += num # horizontal for part 1
    elseif command == "up"
        aim -= num # horizontal for part 1 
    else
        println("*** ERROR ***")
    end
end
println(horizontal * depth) # 2101031224

2101031224


## Day 3

### Part 1

Each bit in the gamma rate can be determined by finding the most common bit in the corresponding position of all numbers in the diagnostic report.

The epsilon rate is calculated in a similar way; rather than use the most common bit, the least common bit from each position is used.

Use the binary numbers in your diagnostic report to calculate the gamma rate and epsilon rate, then multiply them together. What is the power consumption of the submarine? (Be sure to represent your answer in decimal, not binary.)

In [33]:
# Avoid reading entire file into memory
# - read first line to find number of bits in each line
# - track total number of lines and sum of bits in each position

# file = "day3_test.txt" # 198
file = "day3.txt" # 4191876

f = open(file, "r")
numbits = length(readline(f))
close(f)

numlines = 0
numones = zeros(Int, numbits)
lines = eachline(file)
for line in lines
    numlines += 1
    for (i, char) in enumerate(line)
        numones[i] += (char == '1')
    end
end

gamma = epsilon = ""
for n in numones
    n / numlines >= 0.5 ? gamma*="1" : gamma*="0"
    n / numlines >= 0.5 ? epsilon*="0" : epsilon*="1"
end


gammadec = parse(Int, gamma, base = 2)
epsilondec = parse(Int, epsilon, base = 2)
gammadec * epsilondec

4191876

### Part 2

Successive filtering

In [45]:
# Avoid reading entire file into memory
# - read first line to find number of bits in each line
# - track total number of lines and sum of bits in each position

# file = "day3_test.txt" # 198
file = "day3.txt" # 4191876

function bitcounter(lines)
    # take an array of strings, each with a binary
    numbits = length(lines[1])
    numlines = 0
    numones = zeros(Int, numbits)
    for line in lines
        numlines += 1
        for (i, char) in enumerate(line)
            numones[i] += (char == '1')
        end
    end
    
    gamma = epsilon = ""
    for n in numones
        n / numlines >= 0.5 ? gamma*="1" : gamma*="0"
        n / numlines >= 0.5 ? epsilon*="0" : epsilon*="1"
    end
    
    gammadec = parse(Int, gamma, base = 2)
    epsilondec = parse(Int, epsilon, base = 2)
    return(
        numlines = numlines,
        numbits = numbits,
        numones = numones,
        gamma = gamma,
        epsilon = epsilon,
        prod = gammadec * epsilondec
    )
end


result = bitcounter(readlines(file))
result[:prod]


4191876